# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [1]:
import numpy as np
import pandas as pd
import random as rnd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [2]:
read_data = open('Data\Voorbeeld_clusterdata.txt')
data = read_data.read()
data = data.splitlines()
read_data.close()

read_results = open('Data\Voorbeeld_clusterresult.txt')
results = read_results.read()
results = results.splitlines()
read_results.close()

def convert(array):
    line_data = []

    for line in array:
        segment = []
        for elem in line.split():
            segment.append(float(elem))
        line_data.append(segment)
    
    return line_data

cordinates = ['z','y','x','w','v','u','t','s']
df_data = pd.DataFrame(convert(data),columns=['ID','z','y','x','w','v','u','t','s']).set_index('ID')
df_results = pd.DataFrame(convert(results), columns=['ID','label']).set_index('ID')
df_clusterd = df_data.copy().drop(columns=cordinates)

df_data.head()

,z,y,x,w,v,u,t,s
ID,,,,,,,,
846160.0,0.388,0.329,0.690,0.900,0.626,0.621,0.399,0.370
820434.0,-0.296,-0.503,-0.454,-0.868,-0.721,-0.918,-0.486,-0.582
849103.0,-0.246,-0.935,-0.277,-0.175,-0.278,-0.075,-0.236,-0.417
846353.0,0.769,0.929,0.977,1.095,1.058,0.864,0.689,0.492
848613.0,-0.365,-0.760,-0.305,-0.293,-0.364,-0.155,-0.472,-0.606


## Cluster methods
### K-means (KMCA):

uitleggen hoe het werkt

In [3]:
class KMCA:
    def __init__(self, k=6, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def set_k(K):
        self.k = K

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        def pred(self, data):
            distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
            classification = distances.index(min(distances))
            return classification
        
        self.pred_classifications = list()
        
        if np.isfortran(data):
            for element in data:
                self.pred_classifications.append(pred(self, element))
        else:
            self.pred_classifications.append(pred(self, data))
            
        return self.pred_classifications

### Own cluster method:

## Output

* get cluster labels
* save labels in text file

### get cluster labels:

In [4]:
kmca = KMCA()
kmca.fit(df_data.values)
df_clusterd['KMCA'] = kmca.predict(df_data.values)

df_clusterd.head()

,KMCA
ID,
846160.0,0
820434.0,4
849103.0,2
846353.0,3
848613.0,2


### save results:

In [5]:
output = open('Data\clusterresultaten.txt', 'w')
output.writelines(df_clusterd.to_csv(sep=' ', index=True, header=False)) ## er zitten enters tussen de regels
output.close()